# Global indicator project
## Create sausage buffer 

This notebook creates sausage buffer to determine local walkable neighborhood (1600m sample points + 50m walkable netowrk buffer)



In [1]:
# Libraries used for OSMnx analyses and output
import networkx as nx
import time 
import osmnx as ox
import matplotlib.pyplot as plt
import numpy as np
import requests
import pandas as pd
import geopandas as gpd
import fiona

from descartes import PolygonPatch
from shapely.geometry import shape,Point, LineString, Polygon


ox.config(use_cache=True, log_console=True)

### Set up project config

In [6]:
place = 'phoenix' 

region = 'Arizona, USA' # study region name

studyregion = 'Phoenix, Arizona, USA'

suffix = '_201905' # output data time

buffer_dist = 1e4 #study region buffer 10km

point_dist = 30 #sample point interval distance

network_type='walk'

OSM_folder = '../data/OSM'

G_filename = 'Phoenix, Arizona, USA_walk_201905.graphml'

point_filepath = '../data/OSM/phoenix_testing_sample_points_201905/phoenix_testing_sample_points_201905.shp'

## Load pedestrain street network and sample points from local folder

In [7]:
G = ox.load_graphml(filename=G_filename, folder=OSM_folder)
G_proj = ox.project_graph(G)
ox.save_graphml(G_proj, filename='{studyregion}_proj_{network_type}{suffix}.graphml'.format(
        studyregion = place, network_type=network_type, suffix = suffix), folder=OSM_folder)


In [ ]:
G_proj = ox.load_graphml(filename='{studyregion}_proj_{network_type}{suffix}.graphml'.format(
        studyregion = place, network_type=network_type, suffix = suffix), folder=OSM_folder)

In [9]:
# load sample point dataframe
sample_points = gpd.GeoDataFrame.from_file(point_filepath)

In [10]:
# create list of sample points to iterate over
point_locations = []

for point in sample_points.geometry: 
    point = (point.x, point.y)
    point_locations = point_locations + [point]

In [95]:
orig_point = point_locations[0:1]
buffer=50
length = 1600
intersection_tolerance = 15

sausagebuffers = []
for point in orig_point:
    orig_node = ox.get_nearest_node(G_proj, point, return_dist=True)
    subgraph = nx.ego_graph(G, orig_node[0], radius=length, distance='length')
    subgraph_proj = ox.project_graph(subgraph)
    subgraph_gdf = ox.graph_to_gdfs(subgraph_proj, nodes=False, edges=True, fill_edge_geometry=True)
    buffer = subgraph_gdf.geometry.buffer(buffer)
    buffer_uu = buffer.geometry.unary_union

## Create sausage buffer

In [11]:
# create sausage buffer
def create_sausage_buffer(G_proj, orig_point, buffer=50, length = 1600, intersection_tolerance = 15):
    # locate closest node on network to 
    orig_node = ox.get_nearest_node(G_proj, orig_point, return_dist=True)
    subgraph = nx.ego_graph(G, orig_node[0], radius=length, distance='length')
    # We project the subgraph to UTM so we can use metres as a unit when buffering
    subgraph_proj = ox.project_graph(subgraph)
    # create buffer
    subgraph_gdf = ox.graph_to_gdfs(subgraph_proj, nodes=False, edges=True, fill_edge_geometry=True)
    buffer = subgraph_gdf.geometry.buffer(buffer)
    buffer_uu = buffer.geometry.unary_union
    return([subgraph_proj, buffer_uu]) #output is list of a sample point graph (1600m), and buffer polygon

In [111]:
start = time.time()
task = 'Buffer network for {} sample points'.format(len(point_locations))
sausagebuffers = []
for point in point_locations:
    sausagebuffers.append(create_sausage_buffer(G_proj, point))
print('Completed task "{}" in {:,.2f} seconds'.format(task,time.time() - start)) 

Completed task "Buffer network for 100 sample points" in 518.95 seconds


## Get sample point neighborhood stats

In [12]:
# Local neighbourhood analysis
def analyse_local_nh(G_proj, orig_point):
    buffer = create_sausage_buffer(G_proj, orig_point)
    orig_node = ox.get_nearest_node(G_proj, orig_point, return_dist=True)
    #get stats
    area_sqm = buffer[1].area
    area_sqkm = area_sqm*1e-06
    stats = ox.basic_stats(buffer[0], area=area_sqm, clean_intersects=True, circuity_dist='euclidean')
    return({
             'area_sqkm':area_sqkm,
             'stats':stats,
             'origin_node_snap_dist': orig_node[1]})
     

In [16]:
# initialise a list
start = time.time()
task = "Calculate local neighbourhood measures for {} points".format(len(point_locations))
nh_estimates = []
for point in point_locations:
    nh_estimates.append(analyse_local_nh(G_proj, point))
    
# area in sqkm
area_km = [x['area_sqkm'] for x in nh_estimates]
# clean intersection density per sqkm
clean_intersection_count = [x['stats']['clean_intersection_count'] for x in nh_estimates]
# clean intersection density
clean_intersection_density_km = [x['stats']['clean_intersection_density_km'] for x in nh_estimates]
# dist to the nodes
origin_node_snap_dist = [x['origin_node_snap_dist'] for x in nh_estimates]

#load sample point dataframe and record new stats
sample_points['area_km'] = area_km
sample_points['clean_intersection_count'] = clean_intersection_count
sample_points['clean_intersection_density_km']= clean_intersection_density_km
sample_points['origin_node_snap_dist']= origin_node_snap_dist

print('Completed task "{}" in {:,.2f} seconds'.format(task,time.time() - start)) 
  

Completed task "Calculate local neighbourhood measures for 100 points" in 650.51 seconds


In [17]:
sample_points.columns

Index(['index', 'Series', 'access', 'area', 'bridge', 'highway', 'junction',
       'key', 'lanes', 'length', 'maxspeed', 'name', 'oneway', 'osmid', 'ref',
       'service', 'tunnel', 'u', 'v', 'width', 'points', 'geometry', 'area_km',
       'clean_intersection_count', 'clean_intersection_density_km',
       'origin_node_snap_dist'],
      dtype='object')

In [18]:
ox.save_gdf_shapefile(sample_points, filename='{studyregion}_sample_points_stats{suffix}'.format(
        studyregion = place, suffix = suffix), folder=OSM_folder)